In [60]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df_aps = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_phc_only_imed.parquet')

df_otc = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_otc_only_imed.parquet')

df_aih = pd.read_parquet('/Users/julianeoliveira/Documents/github/Bivariate_Anomaly_Detection_Primary_Health_Care_Drug_Selling_ILI_surveillance/Results/data_manuscript_warning_aih_imed.parquet')


In [32]:
df_aih = df_aih[df_aih.year_week >= '2022-47']

In [33]:
df_aps = df_aps[['nm_uf', 'co_imed', 'Nome_imed', 'year_week', 'year_week_ts',
       'epidemi_cal_start', 'epidemi_cal_end',  'out_of_limits_above_phc_only']]

df_otc = df_otc[[ 'co_imed',  'year_week', 'out_of_limits_above_otc_only']]

In [36]:
lst = []

for value in df_aih.co_imed.unique():
    set_aih = df_aih[df_aih.co_imed == value]
    set_aps = df_aps[df_aps.co_imed == value][['nm_uf', 'Nome_imed', 'year_week', 'year_week_ts',
       'epidemi_cal_start', 'epidemi_cal_end',  'out_of_limits_above_phc_only']]
    set_otc = df_otc[df_otc.co_imed == value][['year_week', 'out_of_limits_above_otc_only']]
    
    
    set_aih = set_aih.merge(set_aps, on = 'year_week', how = 'left')
    
    set_aih = set_aih.merge(set_otc, on = 'year_week', how = 'left')
    
    lst.append(set_aih)

In [39]:
df = pd.concat(lst)

In [43]:
df.head(5)

,co_imed,year_week,warning_aih,n,warning_aih_without_isolated,warning_aih_corect_with_consec,warning_final_aih,nm_uf,Nome_imed,year_week_ts,epidemi_cal_start,epidemi_cal_end,out_of_limits_above_phc_only,out_of_limits_above_otc_only
0,110001,2022-47,0,47,0,0,0,Rondônia,Porto Velho,2022-11-21,2022-11-20,2022-11-26,0,0
1,110001,2022-48,0,51,0,0,0,Rondônia,Porto Velho,2022-11-28,2022-11-27,2022-12-03,0,0
2,110001,2022-49,0,51,0,0,0,Rondônia,Porto Velho,2022-12-05,2022-12-04,2022-12-10,0,0
3,110001,2022-50,0,59,0,0,0,Rondônia,Porto Velho,2022-12-12,2022-12-11,2022-12-17,0,0
4,110001,2022-51,0,32,0,0,0,Rondônia,Porto Velho,2022-12-19,2022-12-18,2022-12-24,0,0


In [44]:
# Feature Engineering: Create lagged features for OTC and APS
def create_lagged_features(df, columns, lags):
    for col in columns:
        for lag in range(1, lags + 1):
            df[f"{col}_lag{lag}"] = df[col].shift(lag)
    return df

In [ ]:
# Define the number of lag days you want to analyze
num_lags = 3  # You can adjust this based on your needs
df2 = create_lagged_features(df, ["out_of_limits_above_otc_only", "out_of_limits_above_phc_only"], num_lags)

In [45]:
lst = []
num_lags = 3

for value in df.co_imed.unique():
    set_reg = df[df.co_imed == value]
    
    set_reg = create_lagged_features(set_reg, ["out_of_limits_above_otc_only", "out_of_limits_above_phc_only"], num_lags)
    
    set_reg = set_reg.dropna()
    
    lst.append(set_reg)

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

...

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_lag{lag}"] = df[col].shift(lag)
<ipython-input-44-1a87b315952d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [48]:
res = pd.concat(lst)

In [55]:
# Select features for clustering
features = [col for col in res.columns if col.startswith("out_of_limits_above_otc") or col.startswith("out_of_limits_above_phc")]
X = res[features]

# Option 1: K-Modes Clustering (Categorical Data)
k_modes = KModes(n_clusters=3, init='Huang', n_init=5, verbose=1)
res["cluster_kmodes"] = k_modes.fit_predict(X)


Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3350, cost: 62165.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 7324, cost: 58192.0
Run 2, iteration: 2/100, moves: 1992, cost: 58192.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 9540, cost: 59993.0
Run 3, iteration: 2/100, moves: 3926, cost: 59993.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 4383, cost: 60163.0
Run 4, iteration: 2/100, moves: 1044, cost: 60163.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 6302, cost: 64868.0
Best run was number 2


In [58]:
res.cluster_kmodes.value_counts()

0    27919
1     8774
2     7019
Name: cluster_kmodes, dtype: int64

In [74]:
set_reg = res[res.co_imed == 110001]

In [83]:
set_reg.groupby(['cluster_kmodes'])[['warning_final_aih','out_of_limits_above_phc_only','out_of_limits_above_otc_only']].sum().reset_index()

,cluster_kmodes,warning_final_aih,out_of_limits_above_phc_only,out_of_limits_above_otc_only
0,0,0,6,9
1,1,0,3,3
2,2,1,18,4


In [79]:
set_reg[set_reg.cluster_kmodes == 2]

,co_imed,year_week,warning_aih,n,warning_aih_without_isolated,warning_aih_corect_with_consec,warning_final_aih,nm_uf,Nome_imed,year_week_ts,...,epidemi_cal_end,out_of_limits_above_phc_only,out_of_limits_above_otc_only,out_of_limits_above_otc_only_lag1,out_of_limits_above_otc_only_lag2,out_of_limits_above_otc_only_lag3,out_of_limits_above_phc_only_lag1,out_of_limits_above_phc_only_lag2,out_of_limits_above_phc_only_lag3,cluster_kmodes
17,110001,2023-12,0,56,0,0,0,Rondônia,Porto Velho,2023-03-20,...,2023-03-25,1,0,0.0,1.0,1.0,1.0,1.0,1.0,2
18,110001,2023-13,0,54,0,0,0,Rondônia,Porto Velho,2023-03-27,...,2023-04-01,1,1,0.0,0.0,1.0,1.0,1.0,1.0,2
19,110001,2023-14,0,74,0,0,0,Rondônia,Porto Velho,2023-04-03,...,2023-04-08,0,0,1.0,0.0,0.0,1.0,1.0,1.0,2
20,110001,2023-15,1,101,1,1,1,Rondônia,Porto Velho,2023-04-10,...,2023-04-15,1,0,0.0,1.0,0.0,0.0,1.0,1.0,2
21,110001,2023-16,1,98,1,1,0,Rondônia,Porto Velho,2023-04-17,...,2023-04-22,1,0,0.0,0.0,1.0,1.0,0.0,1.0,2
22,110001,2023-17,1,110,1,1,0,Rondônia,Porto Velho,2023-04-24,...,2023-04-29,1,0,0.0,0.0,0.0,1.0,1.0,0.0,2
23,110001,2023-18,1,107,1,1,0,Rondônia,Porto Velho,2023-05-01,...,2023-05-06,1,0,0.0,0.0,0.0,1.0,1.0,1.0,2
24,110001,2023-19,1,100,1,1,0,Rondônia,Porto Velho,2023-05-08,...,2023-05-13,1,0,0.0,0.0,0.0,1.0,1.0,1.0,2
25,110001,2023-20,1,110,1,1,0,Rondônia,Porto Velho,2023-05-15,...,2023-05-20,1,0,0.0,0.0,0.0,1.0,1.0,1.0,2
26,110001,2023-21,1,99,1,1,0,Rondônia,Porto Velho,2023-05-22,...,2023-05-27,1,0,0.0,0.0,0.0,1.0,1.0,1.0,2


In [67]:
res.groupby(['cluster_kmodes'])[['warning_final_aih', 'out_of_limits_above_phc_only', 'out_of_limits_above_otc_only', 'out_of_limits_above_otc_only_lag1', 'out_of_limits_above_otc_only_lag2', 'out_of_limits_above_otc_only_lag3']].sum().reset_index()


,cluster_kmodes,warning_final_aih,out_of_limits_above_phc_only,out_of_limits_above_otc_only,out_of_limits_above_otc_only_lag1,out_of_limits_above_otc_only_lag2,out_of_limits_above_otc_only_lag3
0,0,157,2846,4756,2708.0,1995.0,3117.0
1,1,100,2120,3643,6317.0,7479.0,6474.0
2,2,67,5410,2537,1825.0,1306.0,1133.0


In [ ]:



# Option 2: DBSCAN (For discovering natural density-based clusters)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
dbscan = DBSCAN(eps=0.5, min_samples=5)
df["cluster_dbscan"] = dbscan.fit_predict(X_scaled)

# Evaluate clusters (Optional)
if len(set(df["cluster_kmodes"])) > 1:
    silhouette = silhouette_score(X, df["cluster_kmodes"], metric='hamming')
    print("Silhouette Score (K-Modes):", silhouette)

# Analyzing Clusters: Identify which cluster best anticipates hospitalization
cluster_summary = df.groupby("cluster_kmodes")["Hospitalization"].mean()
print("Cluster Hospitalization Rates:")
print(cluster_summary)

# Save the results
df.to_csv("clustered_data.csv", index=False)
